In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# service_fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx') \
#     .rename(columns = {'SPN': 'spn', 'J1939 FMI': 'fmi', 'Lamp Device': 'LampDevice', 'Cummins Description': 'Description'}) \
#     [['spn', 'fmi', 'Description']] \
#     .drop_duplicates()
# service_fault_codes['Description'] = service_fault_codes['Description'].str.split(' - ').str[0]
# service_fault_codes = faults_diagnostics.merge(service_fault_codes, how = 'left', on = ['spn', 'fmi'])[['spn', 'fmi', 'eventDescription', 'Description']]
# service_fault_codes.value_counts(dropna = False).to_frame('Count').to_csv('../data/service_fault_codes.csv')

In [ ]:
service_fault_codes_curated = pd.read_csv('../data/service_fault_codes_curated.csv').drop(columns = 'Count')

In [ ]:
# Read in faults_diagnostics data
faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv', index_col = 'EventTimeStamp', parse_dates = ['EventTimeStamp'])
faults_diagnostics = faults_diagnostics.loc[faults_diagnostics['active'] == True].drop(columns = 'active').drop_duplicates()

columns = [
    'EventTimeStamp', 'ESS_Id', 'EquipmentID', 'spn', 'fmi', 'Component', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlActive', 
    'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 
    'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake', 
    'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 
]

faults_diagnostics = faults_diagnostics.reset_index().merge(service_fault_codes_curated, how = 'left', on = ['spn', 'fmi'])[columns].set_index('EventTimeStamp')
faults_diagnostics = pd.get_dummies(faults_diagnostics, prefix = '', prefix_sep = '', columns = ['Component'], drop_first = True)

In [ ]:
agg_dict = {
    'Aftertreatment System': 'sum', 
    'Ambient Air Temperature': 'sum',
    'Anti-Lock Braking (ABS) Controller': 'sum',
    'Auxiliary Input/Output': 'sum',
    'Auxiliary Temperature Sensor Input Circuit': 'sum',
    'Barometric Pressure Sensor': 'sum',
    'Battery': 'sum', 
    'Crankcase': 'sum',
    'DAS Module to ECM Communication': 'sum',
    'ECU Power Output Supply Voltage': 'sum',
    'Electric Lift Pump': 'sum', 
    'Electrical Charging System Voltage': 'sum',
    'Electronic Fuel Injection Control Valve': 'sum',
    'Electronic Throttle Control': 'sum', 
    'Engine Air Filter': 'sum',
    'Engine Brake': 'sum',
    'Engine Charge Air Cooler Outlet Temperature': 'sum', 
    'Engine Control Module': 'sum',
    'Engine Coolant': 'sum', 
    'Engine Cooling System': 'sum',
    'Engine Diesel Particulate Filter': 'sum',
    'Engine Exhaust Back Pressure': 'sum',
    'Engine Fuel': 'sum', 
    'Engine Injector Metering Rail': 'sum',
    'Engine Intake Air Heater': 'sum', 
    'Engine Misfire': 'sum', 
    'Engine Oil': 'sum',
    'Engine Position Sensor': 'sum', 
    'Engine Protection Torque Full Derate': 'sum',
    'Engine Protection Torque Partial Derate': 'sum',
    'Engine Speed': 'sum',
    'Engine Starter Motor Relay': 'sum', 
    'Engine Total Idle Hours': 'sum', 
    'Exhaust Gas': 'sum',
    'Exhaust Gas Recirculation System': 'sum', 
    'Fan Control': 'sum',
    'Fuel Control Module': 'sum',
    'High Pressure Common Rail Fuel Pressure Relief Valve': 'sum',
    'Indicator Lamp': 'sum',
    'Injector Metering Rail': 'sum', 
    'Injector Solenoid': 'sum',
    'Intake Manifold': 'sum', 
    'Intake Throttle': 'sum', 
    'J1939 Network': 'sum',
    'Real Time Clock': 'sum', 
    'Sensor Supply': 'sum', 
    'Steering Wheel Angle Sensor': 'sum',
    'Total Vehicle Distance': 'sum',
    'Transmission Communications Failure Idle Torque Limit': 'sum',
    'Transmission Oil': 'sum', 
    'Turbocharger': 'sum', 
    'Unknown': 'sum',
    'Variable Geometry Turbocharger': 'sum', 
    'Vehicle Identification Number': 'sum',
    'Wheel-Based Vehicle Speed': 'sum'
}

rolling_21D = faults_diagnostics.groupby(['EquipmentID']).rolling('21D', closed = 'left').agg(agg_dict).reset_index().set_index('EventTimeStamp')

In [ ]:
component_cols = [
    'Aftertreatment System',
    'Ambient Air Temperature',
    'Anti-Lock Braking (ABS) Controller',
    'Auxiliary Input/Output',
    'Auxiliary Temperature Sensor Input Circuit',
    'Barometric Pressure Sensor',
    'Battery', 
    'Crankcase',
    'DAS Module to ECM Communication',
    'ECU Power Output Supply Voltage',
    'Electric Lift Pump', 
    'Electrical Charging System Voltage',
    'Electronic Fuel Injection Control Valve',
    'Electronic Throttle Control', 
    'Engine Air Filter',
    'Engine Brake',
    'Engine Charge Air Cooler Outlet Temperature', 
    'Engine Control Module',
    'Engine Coolant', 
    'Engine Cooling System',
    'Engine Diesel Particulate Filter',
    'Engine Exhaust Back Pressure',
    'Engine Fuel', 
    'Engine Injector Metering Rail',
    'Engine Intake Air Heater', 
    'Engine Misfire', 
    'Engine Oil',
    'Engine Position Sensor', 
    'Engine Protection Torque Full Derate',
    'Engine Protection Torque Partial Derate',
    'Engine Speed',
    'Engine Starter Motor Relay', 
    'Engine Total Idle Hours', 
    'Exhaust Gas',
    'Exhaust Gas Recirculation System', 
    'Fan Control',
    'Fuel Control Module',
    'High Pressure Common Rail Fuel Pressure Relief Valve',
    'Indicator Lamp',
    'Injector Metering Rail', 
    'Injector Solenoid',
    'Intake Manifold', 
    'Intake Throttle', 
    'J1939 Network',
    'Real Time Clock', 
    'Sensor Supply', 
    'Steering Wheel Angle Sensor',
    'Total Vehicle Distance',
    'Transmission Communications Failure Idle Torque Limit',
    'Transmission Oil', 
    'Turbocharger', 
    'Unknown',
    'Variable Geometry Turbocharger', 
    'Vehicle Identification Number',
    'Wheel-Based Vehicle Speed'  
]

final_df = pd.concat([faults_diagnostics.drop(columns = component_cols).sort_values(['EquipmentID', 'EventTimeStamp']), rolling_21D.drop(columns = 'EquipmentID')], axis = 1)

In [ ]:
final_df.to_csv('../data/final_df.csv')

In [ ]:
# faults_diagnostics['PartialDerate'] = (faults_diagnostics['spn'] == 1569)
# faults_diagnostics['FullDerate'] = (faults_diagnostics['spn'] == 5246)

# rolling_7D = faults_diagnostics.loc[::-1].groupby('EquipmentID', as_index = False)[['PartialDerate', 'FullDerate']].rolling('7D').sum()[::-1]
# rolling_7D.loc[rolling_7D['PartialDerate'] > 0, 'PartialDerate'] = 1
# rolling_7D.loc[rolling_7D['FullDerate'] > 0, 'FullDerate'] = 1

# rolling_7D['PartialDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['PartialDerate'].shift() != rolling_7D['PartialDerate'])).cumsum()
# rolling_7D['FullDerate'] = ((rolling_7D['EquipmentID'].shift() != rolling_7D['EquipmentID']) | (rolling_7D['FullDerate'].shift() != rolling_7D['FullDerate'])).cumsum()

# faults_diagnostics = faults_diagnostics.drop(columns = ['PartialDerate', 'FullDerate'])
# faults_diagnostics = faults_diagnostics.merge(rolling_7D, on = ['EventTimeStamp', 'EquipmentID'])
# faults_diagnostics = faults_diagnostics.sort_values(['EquipmentID', 'EventTimeStamp']).drop_duplicates()

In [ ]:
# partial_derate_clusters = faults_diagnostics.loc[faults_diagnostics['PartialDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 1569, 'PartialDerate'])]
# partial_derate_mask = partial_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(1569).cumsum().eq(0)).to_list()
# partial_derates = partial_derate_clusters[partial_derate_mask].reset_index()

# partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
# partial_derates['DeltaT'] = partial_derates.loc[::-1].groupby(['EquipmentID', 'PartialDerate'])['DeltaT'].cumsum()[::-1]
# partial_derates = partial_derates.loc[partial_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'PartialDerate', 'spn'], keep = 'last')
# partial_derates['DeltaT'] = partial_derates['DeltaT'].fillna(0)

# partial_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

In [ ]:
# full_derate_clusters = faults_diagnostics.loc[faults_diagnostics['FullDerate'].isin(faults_diagnostics.loc[faults_diagnostics['spn'] == 5246, 'FullDerate'])]
# full_derate_mask = full_derate_clusters.groupby('EquipmentID')['spn'].apply(lambda x: x.shift().eq(5246).cumsum().eq(0)).to_list()
# full_derates = full_derate_clusters[full_derate_mask].reset_index()

# full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['EventTimeStamp'].diff().dt.total_seconds().div(3600)[::-1].abs()
# full_derates['DeltaT'] = full_derates.loc[::-1].groupby(['EquipmentID', 'FullDerate'])['DeltaT'].cumsum()[::-1]
# full_derates = full_derates.loc[full_derates['DeltaT'] != 0].set_index('EventTimeStamp').drop_duplicates(subset = ['EquipmentID', 'FullDerate', 'spn'], keep = 'last')
# full_derates['DeltaT'] = full_derates['DeltaT'].fillna(0)

# full_derates.groupby('spn')['DeltaT'].describe().sort_values('count', ascending = False)

In [ ]:
# event_timestamps = faults_diagnostics.reset_index()[['EventTimeStamp','EquipmentID']].drop_duplicates()
# event_timestamps['DeltaT'] = event_timestamps.groupby('EquipmentID')['EventTimeStamp'].diff(-1).dt.total_seconds().div(3600).abs()
# event_timestamps = faults_diagnostics.merge(event_timestamps, on = ['EventTimeStamp', 'EquipmentID'])
# event_timestamps = event_timestamps[event_timestamps['DeltaT'] <= 168]

In [ ]:
# fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
# sns.histplot(event_timestamps['DeltaT'])
# plt.xticks([0,24,48,72,96,120,144,168])
# ax.set_yscale('log', base = 2)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
# sns.histplot(event_timestamps[event_timestamps['spn'] == 1569]['DeltaT'])
# plt.xticks([0,24,48,72,96,120,144,168])
# ax.set_yscale('log', base = 2)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize = (8, 5), dpi = 300)
# sns.histplot(event_timestamps[event_timestamps['spn'] == 5246]['DeltaT'])
# plt.xticks([0,24,48,72,96,120,144,168])
# ax.set_yscale('log', base = 2)
# plt.show()

In [ ]:
# spn_crosstab = faults_diagnostics
# spn_crosstab['prev_spn'] = spn_crosstab.groupby(['EquipmentID'])['spn'].shift(-1)
# spn_crosstab = spn_crosstab.dropna(subset = 'prev_spn')
# spn_crosstab = pd.crosstab(spn_crosstab['spn'], spn_crosstab['prev_spn'].astype(int))
# spn_crosstab.style.background_gradient(cmap = 'Blues')